In [ ]:
!pip install requests

In [ ]:
import requests
from datetime import datetime
from dataclasses import dataclass

DOMAIN = 'myschoolmenus.com'


@dataclass
class RequestParams:
    path: str = None
    headers: dict = None
    exception_message: str = None


class Request:
    def __init__(self):
        pass

    @staticmethod
    def url(path) -> str:
        """
        Get the URL for the API.

        :return: URL for the API.
        :rtype: str
        """

        return f"https://{DOMAIN}{path}"

    @staticmethod
    def get(params: RequestParams) -> dict:
        """
        Get a response from the API.

        :param params: Request parameters.

        :return: Response from API.
        :rtype: dict
        """

        url = f"{Request.url(params.path)}"
        response = requests.get(url=url, headers=params.headers)
        if response.status_code != 200:
            raise ValueError(
                f"Endpoint {url} returned status code {response.status_code}: {response.reason}"
            )
        try:
            json = response.json()
            if not json['data']:
                raise ValueError(
                    params.exception_message
                )
        except requests.exceptions.JSONDecodeError:
            raise ValueError(
                f"Unable to decode JSON response"
            )
        return response.json()


class Menus:
    def __init__(self):
        self.path = '/api/organizations'

    def get(self, district_id: int, site_id: int = None, menu_id: int = None, date: datetime.date = None) -> dict:
        """
        Get a menu by district ID, menu ID, and optionally a date.
        
        This can be used to fetch any type of menu (breakfast, lunch, etc.) by providing the appropriate menu_id.

        :param district_id: District ID.
        :param site_id: Site ID.
        :param menu_id: Menu ID.
        :param date: Date of menu.

        :return: District menu.
        :rtype: dict
        """

        exception_message = f"No menu found for district {district_id}"
        if site_id:
            path = self.path + f"/{district_id}/sites/{site_id}"
        elif menu_id:
            path = self.path + f"/{district_id}/menus/{menu_id}"
            if date:
                path = path + f"/year/{date.strftime('%Y')}/month/{date.strftime('%m')}/date_overwrites"
        else:
            path = self.path
        return Request.get(RequestParams(
            path=path,
            exception_message=exception_message + f", menu {menu_id}{f', and date {date}' if date else ''}" if menu_id else exception_message
        ))

    @staticmethod
    def menu_months(menu: dict) -> list[datetime.date]:
        """
        Get a list of months with available menus.  Does not include all available prior months.

        :param menu: Menu to get months.

        :return: List of months available for a menu.
        :rtype: list
        """

        return [datetime.fromisoformat(date) for date in menu['data']['published_months']]


class Organizations:
    def __init__(self):
        self.path = '/api/organizations'

    def get(self, organization_id: int = None) -> dict:
        """
        Get an organization by organization ID, or return all organizations.

        :param organization_id: Organization ID.

        :return: District organization.
        :rtype: dict
        """
        return Request.get(RequestParams(
            path=self.path + f"/{organization_id}" if organization_id else self.path,
            exception_message=f"No organization found for organization {organization_id}" if organization_id else ""
        ))


class Sites:
    def __init__(self):
        self.path = 'api/organizations'

    def get(self, district_id: int, site_id: int) -> dict:
        """
        Get a site for a given district.

        :param district_id: District ID.
        :param site_id: Site ID.

        :return: District site.
        :rtype: dict
        """
        return Request.get(RequestParams(
            path=self.path + f"/{district_id}/sites/{site_id}",
            exception_message=f"No site found for district {district_id}" + f" and site {site_id}"
        ))

In [ ]:
import json
import uuid
from datetime import datetime, time, timedelta

class Calendar:
    def __init__(self, default_breakfast_time=time(8, 0), default_lunch_time=time(12, 0)):
        """
        Initialize the Calendar with default times for meals.

        :param default_breakfast_time: Default time for breakfast events (default: 8:00 AM)
        :param default_lunch_time: Default time for lunch events (default: 12:00 PM)
        """
        self.default_breakfast_time = default_breakfast_time
        self.default_lunch_time = default_lunch_time

    def events(self, menu: json, menu_type: str = "lunch", include_time: bool = False) -> list:
        """
        Generate a list of events from a menu

        :param menu: json menu.
        :param menu_type: Type of menu ("breakfast" or "lunch").
        :param include_time: Whether to include time in events.

        :return: List of events.
        :rtype: list
        """
        event_list = []
        menu_data = menu['data']
        if not menu_data:
            raise ValueError(
                f"Missing menu data."
            )

        # Set the event time based on menu type
        event_time = None
        if include_time:
            if menu_type.lower() == "breakfast":
                event_time = self.default_breakfast_time
            else:  # Default to lunch time for any other menu type
                event_time = self.default_lunch_time

        for entry in menu_data:
            if entry is None:
                continue
            
            try:
                # Use menu_type from parameter to determine which prefix to use
                prefix = "L: " if menu_type.lower() == "lunch" else "B: "
                
                # Process each item in the menu
                recipe_count = 0
                category_count = 0
                summary = ""
                description_parts = []
                
                for item in json.loads(entry['setting'])['current_display']:
                    if item['type'] == 'recipe' and recipe_count == 0:
                        recipe_count += 1
                        summary = f"{prefix}{item['name']}"
                    
                    if item['type'] == 'category' and category_count == 0:
                        category_count += 1
                        description_parts.append(f"{item['name']}:")
                    elif item['type'] == 'category':
                        description_parts.append("")  # Empty line
                        description_parts.append(f"{item['name']}:")
                    else:
                        description_parts.append(item['name'])
                
                if summary == '':
                    continue
                
                # Create event dictionary instead of using icalendar library
                event = {
                    'summary': summary,
                    'description': description_parts,
                    'uid': str(uuid.uuid4()),
                    'dtstamp': datetime.now(),
                    'transp': 'OPAQUE'
                }
                
                entry_date = datetime.fromisoformat(entry['day']).date()
                
                # Add time to the event if requested
                if include_time and event_time:
                    event_datetime = datetime.combine(entry_date, event_time)
                    event['dtstart'] = event_datetime
                    
                    # Add event duration (30 minutes for breakfast, 45 minutes for lunch)
                    if menu_type.lower() == "breakfast":
                        duration = timedelta(minutes=30)
                    else:
                        duration = timedelta(minutes=45)
                    
                    end_time = event_datetime + duration
                    event['dtend'] = end_time
                else:
                    event['dtstart'] = entry_date
                
                event_list.append(event)
            except KeyError:
                continue

        return event_list

    def combine_calendars(self, calendar_list: list) -> list:
        """
        Combine multiple calendars into a single calendar

        :param calendar_list: List of calendars.

        :return: Combined calendar events.
        :rtype: list
        """
        combined_events = []
        for cal in calendar_list:
            combined_events.extend(cal)
        return combined_events

    def calendar(self, cal_events: list) -> list:
        """
        Generate a calendar from events

        :param cal_events: list of events.
        
        :return: Calendar events.
        :rtype: list
        """
        return cal_events

    def ical(self, events: list) -> str:
        """
        Get the menu as a properly formatted iCal string.

        :param events: List of event dictionaries.

        :return: Properly formatted iCal string.
        :rtype: str
        """
        lines = [
            "BEGIN:VCALENDAR",
            "VERSION:2.0",
            "PRODID:-//My School Menus//EN",
            "CALSCALE:GREGORIAN",
            "METHOD:PUBLISH"
        ]
        
        for event in events:
            lines.append("BEGIN:VEVENT")
            
            # Add summary
            lines.append(f"SUMMARY:{event['summary']}")
            
            # Add start time/date
            if isinstance(event['dtstart'], datetime):
                dt_str = event['dtstart'].strftime("%Y%m%dT%H%M%S")
                lines.append(f"DTSTART:{dt_str}")
            else:
                dt_str = event['dtstart'].strftime("%Y%m%d")
                lines.append(f"DTSTART;VALUE=DATE:{dt_str}")
            
            # Add end time/date if present
            if 'dtend' in event:
                dt_str = event['dtend'].strftime("%Y%m%dT%H%M%S")
                lines.append(f"DTEND:{dt_str}")
            
            # Add timestamp
            dt_str = event['dtstamp'].strftime("%Y%m%dT%H%M%SZ")
            lines.append(f"DTSTAMP:{dt_str}")
            
            # Add UID
            lines.append(f"UID:{event['uid']}")
            
            # Add description with proper folding
            description = "\\n".join(event['description'])
            lines.extend(self._fold_content("DESCRIPTION", description))
            
            # Add transparency
            lines.append(f"TRANSP:{event['transp']}")
            
            lines.append("END:VEVENT")
        
        lines.append("END:VCALENDAR")
        
        # Join with proper line endings for iCalendar (CRLF)
        return "\r\n".join(lines)
    
    @staticmethod
    def _fold_content(property_name, content):
        """
        Properly fold long content according to iCalendar spec (75 chars)
        
        :param property_name: The property name
        :param content: The content to fold
        :return: List of properly folded lines
        """
        # First line includes property name
        first_line = f"{property_name}:{content}"
        if len(first_line) <= 75:
            return [first_line]
            
        # Need to fold
        result = []
        current_line = first_line
        
        while len(current_line) > 75:
            # Add the first 75 chars to the result
            result.append(current_line[:75])
            # The rest becomes the next line, with a space at the beginning
            current_line = " " + current_line[75:]
        
        # Don't forget the remainder
        if current_line:
            result.append(current_line)
            
        return result

    def set_breakfast_time(self, hour: int, minute: int = 0):
        """
        Set the default time for breakfast events.

        :param hour: Hour (0-23)
        :param minute: Minute (0-59)
        """
        self.default_breakfast_time = time(hour, minute)

    def set_lunch_time(self, hour: int, minute: int = 0):
        """
        Set the default time for lunch events.

        :param hour: Hour (0-23)
        :param minute: Minute (0-59)
        """
        self.default_lunch_time = time(hour, minute)

In [ ]:
#@title School and Menu Configuration
#@markdown --- 
#@markdown ### Add your school and menu information below
school_configs = [
    {
        "district_id": 2230,
        "site_id": 14066,
        "lunch_menu_id": 65638,
        "breakfast_menu_id": 90143
    }
]

#@markdown ### Combine all schools into a single calendar file?
combine_calendars = False #@param {type:"boolean"}


In [ ]:
def generate_ics():
    menus = Menus()
    cal = Calendar(
        default_breakfast_time=time(8, 0),
        default_lunch_time=time(12, 0)
    )

    all_events = []
    
    for i, config in enumerate(school_configs):
        district_id = config['district_id']
        site_id = config['site_id']
        lunch_menu_id = config.get('lunch_menu_id')
        breakfast_menu_id = config.get('breakfast_menu_id')

        school_events = []
        
        try:
            if lunch_menu_id:
                lunch_menu = menus.get(district_id=district_id, menu_id=lunch_menu_id, date=datetime.now())
                school_events.extend(cal.events(lunch_menu, menu_type="lunch", include_time=True))
            if breakfast_menu_id:
                breakfast_menu = menus.get(district_id=district_id, menu_id=breakfast_menu_id, date=datetime.now())
                school_events.extend(cal.events(breakfast_menu, menu_type="breakfast", include_time=True))
            
            if not combine_calendars:
                calendar = cal.calendar(school_events)
                ical = cal.ical(calendar)
                with open(f'school-{district_id}-{site_id}-menu-calendar.ics', 'w', newline='') as f:
                    f.write(ical)
            
            all_events.extend(school_events)
            print(f"Successfully generated calendar for school {district_id}-{site_id}")
        except Exception as e:
            print(f"Error generating calendar for school {district_id}-{site_id}: {e}")

    if combine_calendars:
        calendar = cal.calendar(all_events)
        ical = cal.ical(calendar)
        with open('school-menu-calendar.ics', 'w', newline='') as f:
            f.write(ical)
        print("Combined ICS file generated.")
    else:
        print("Individual ICS files generated.")

generate_ics()

## How to Download Your Calendar Files

1.  Click the **folder icon** in the left-hand sidebar to open the file browser.
2.  You should see your generated `.ics` files in the file list.
3.  Click the **three dots** next to the file you want to download and select **Download**.